Enron dataset link below. Extract email network. Find From-To relationship for d3 graph. Consider all emails in the "To" field.

Data: https://www.kaggle.com/wcukierski/enron-email-dataset

In [7]:
import numpy as np
import pandas as pd
import re

In [8]:
PATH = "Resources/"

In [12]:
pd.options.mode.chained_assignment = None

# Huge dataset need to look at smaller section aka chunk and look at the layout

chunk = pd.read_csv(PATH + 'emails.csv', chunksize=5000)
data = next(chunk)

data.info()

# Pick random email to view via number below
print(data.message[981])

data = pd.read_csv(PATH + 'emails.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
file       5000 non-null object
message    5000 non-null object
dtypes: object(2)
memory usage: 78.2+ KB
Message-ID: <31291337.1075855695811.JavaMail.evans@thyme>
Date: Mon, 26 Mar 2001 06:52:00 -0800 (PST)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_June2001\Notes Folders\All documents
X-Origin: Allen-P
X-FileName: pallen.nsf

John,

Mike is fine with signing a new contract (subject to reading the terms, of 
course).  He prefers to set strikes over a 3 month period.  His existing 
contract pays him a retention payment of $55,000 in the next week.  He still 
wants to receive this payment.

Phillip


In [17]:
# Set up dataframe
links = pd.DataFrame(columns=['Date', 'From', 'To'])
links.head()


,Date,From,To


In [18]:
# Time to figure out email addresses
fromRegex = re.compile(r'From:\s(\w)+(\.)?(\w)*@(\w)+.com')
toRegex = re.compile(r'To:\s(\w)+(\.)?(\w)*@(\w)+.com(.*?)\n')
dateRegex = re.compile(r'Date:\s(.*?)\n')
links_index = 0
for i in range(len(data)):
    mf = fromRegex.search(data['message'][i])
    me = toRegex.search(data['message'][i])
    date = dateRegex.search(data['message'][i])
    if mf and me:
        date = date.group().replace('Date: ', '').replace('\n','')
        m_from = mf.group().replace('From: ', '')
        m_to = me.group().replace('To: ', '').replace('\n', '')
        if ',' in m_to:
            addresses = m_to.split(', ')
            for address in addresses:
                if address != '':
                    links.loc[links_index] = [date, m_from, address]
                    links_index += 1
        else: 
            links.loc[links_index] = [date, m_from, m_to]
            links_index += 1

In [19]:
links.Date = pd.to_datetime(links.Date)

In [20]:
links.shape

(614586, 3)

In [21]:
links.to_csv("email-links-all.csv")

In [22]:
from datetime import datetime
str(datetime.now())

'2018-12-12 00:31:37.051598'

In [23]:
links.head()

,Date,From,To
0,2001-05-14 23:39:00,phillip.allen@enron.com,tim.belden@enron.com
1,2001-05-04 20:51:00,phillip.allen@enron.com,john.lavorato@enron.com
2,2000-10-18 10:00:00,phillip.allen@enron.com,leah.arsdall@enron.com
3,2000-10-23 13:13:00,phillip.allen@enron.com,randall.gay@enron.com
4,2000-08-31 12:07:00,phillip.allen@enron.com,greg.piper@enron.com
